In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
# Reads data
features = pd.read_csv("data/features.csv")
target = pd.read_csv("data/target.csv")

In [14]:
# Bins target values
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=4, encode = "onehot-dense", strategy = "quantile")

target_discretized = discretizer.fit_transform(target["G3"].values.reshape(-1, 1))
target_new = []
for row in target_discretized:
    target_new.append(list(row).index(1))

In [15]:
# calculates counts for original data
counts = [0] * 21
for datum in target["G3"].values:
    counts[datum] += 1


In [16]:
# calculates counts for binned data
counts = [0, 0, 0, 0]
for datum in target_new:
    counts[datum] += 1
counts

[230, 153, 367, 294]

In [17]:
# Splits data into training, validation, testing sets
X_train, X_test_val, y_train, y_test_val = train_test_split(features, target, test_size = 0.3,random_state = 1001)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size = 0.5, random_state = 1001)

In [18]:
# Gets final grade for target
t_train = y_train["G3"]
t_val = y_val["G3"]
t_test = y_test["G3"]

In [19]:
# Imports Ridge and accuracy metrics
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Performs Hyperparameter tuning
param_grid = [0.001, 0.01, 0.1, 1, 10, 100]
models = []
for param in param_grid:
    models.append(Ridge(alpha = param).fit(X=X_train, y=t_train))
scores = []
for model in models:
    scores.append(r2_score(t_val, model.predict(X_val)))
best_score = scores[scores.index(max(scores))]
best_param = param_grid[scores.index(max(scores))]
best_model = models[scores.index(max(scores))]

# Saves results
ridge_train_score = r2_score(t_train, best_model.predict(X_train))
ridge_val_score =  r2_score(t_val, best_model.predict(X_val))
ridge_test_score = r2_score(t_test, best_model.predict(X_test))
# Measures accuracy
print("Best Parameter:", best_param)
print("R-squared value for training set: ", r2_score(t_train, best_model.predict(X_train)))
print("R-squared value for testing set: ", r2_score(t_test, best_model.predict(X_test)))

Best Parameter: 100
R-squared value for training set:  0.31458176149234485
R-squared value for testing set:  0.2651854418480495


In [20]:
# Imports Lasso and accuracy metrics
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs hyperparameter tuning
param_grid = [0.001, 0.01, 0.1, 1, 10, 100]
models = []
for param in param_grid:
    models.append(Lasso(alpha = param).fit(X=X_train, y=t_train))
scores = []
for model in models:
    scores.append(r2_score(t_val, model.predict(X_val)))
best_score = scores[scores.index(max(scores))]
best_param = param_grid[scores.index(max(scores))]
best_model = models[scores.index(max(scores))]
# Saves results
lasso_train_score = r2_score(t_train, best_model.predict(X_train))
lasso_val_score =  r2_score(t_val, best_model.predict(X_val))
lasso_test_score = r2_score(t_test, best_model.predict(X_test))
# Measures accuracy
print("Best Parameter:", best_param)
print("R-squared value for training set: ", r2_score(t_train, best_model.predict(X_train)))
print("R-squared value for testing set: ", r2_score(t_test, best_model.predict(X_test)))

Best Parameter: 0.01
R-squared value for training set:  0.3446902144741497
R-squared value for testing set:  0.26472075923764227


In [21]:
# Saves a table for regression results
summary = pd.DataFrame(data = {"Ridge": [ridge_train_score, ridge_val_score, ridge_test_score], "Lasso":[lasso_train_score, lasso_val_score, lasso_test_score]})
summary.index = ["R^2 on training", "R^2 on validation", "R^2 on testing"]
summary.T.to_html("tables/regression.html")

In [22]:
# Splits data into training, valdation, testing sets
X_train, X_test_val, y_train, y_test_val = train_test_split(features, target_new, test_size = 0.3,random_state = 1001)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size = 0.5, random_state = 1001)


In [23]:
# Imports SVC and accuracy metrics
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs hyperparameter tuning
param_grid = [0.001, 0.01, 0.1, 1, 10, 100]
models = []
for param in param_grid:
    models.append(SVC(gamma = param).fit(X=X_train, y=y_train))
scores = []
for model in models:
    scores.append(model.score(X_val, y_val))
# Saves results
best_score = scores[scores.index(max(scores))]
best_param = param_grid[scores.index(max(scores))]
best_model = models[scores.index(max(scores))]


# Measures accuracy
kernel_accuracy_train = best_model.score(X_train, y_train)
kernel_accuracy_val = best_score
kernel_accuracy_test = best_model.score(X_test, y_test)
kernel_scores = [kernel_accuracy_train, kernel_accuracy_val, kernel_accuracy_test]
print("Best Parameter:", best_param)
print("Prediction accuracy on the train data:", f"{kernel_accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{kernel_accuracy_test:.2%}")

Best Parameter: 0.001
Prediction accuracy on the train data: 48.22%
Prediction accuracy on the test data: 36.94%


In [24]:
from sklearn.svm import LinearSVC
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Trains model
model = LinearSVC().fit(X_train, y_train)


# Measures accuracy
linear_accuracy_train = model.score(X_train, y_train)
linear_accuracy_val = model.score(X_val, y_val)
linear_accuracy_test = model.score(X_test, y_test)
linear_scores = [linear_accuracy_train, linear_accuracy_val, linear_accuracy_test]
print("Prediction accuracy on the train data:", f"{linear_accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{linear_accuracy_test:.2%}")

Prediction accuracy on the train data: 43.42%
Prediction accuracy on the test data: 35.03%


In [25]:
# Imports Decision Tree and accuracy metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs hyperparameter tuning
param_grid = [1, 5, 10, 25, 50]
models = []
for param in param_grid:
    models.append(DecisionTreeClassifier(max_depth = param).fit(X=X_train, y=y_train))
scores = []
for model in models:
    scores.append(model.score(X_val, y_val))

# Saves results
best_score = scores[scores.index(max(scores))]
best_param = param_grid[scores.index(max(scores))]
best_model = models[scores.index(max(scores))]

# Performs Decision Tree Classifier
model = DecisionTreeClassifier().fit(X=X_train, y=y_train)

# Measures accuracy
print("Best Parameter:", best_param)
dt_accuracy_train = best_model.score(X_train, y_train)
dt_accuracy_val = best_score
dt_accuracy_test = best_model.score(X_test, y_test)
dt_scores = [dt_accuracy_train, dt_accuracy_val, dt_accuracy_test]
print("Prediction accuracy on the train data:", f"{dt_accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{dt_accuracy_test:.2%}")

Best Parameter: 5
Prediction accuracy on the train data: 56.58%
Prediction accuracy on the test data: 35.03%


In [26]:
import torch

class Layer(torch.nn.Module):
    def __init__(self, inputDim, outputDim, activation):
        
        super(Layer, self).__init__()
        self.inputDim = inputDim
        self.outputDim = outputDim
        self.linear = torch.nn.Linear(inputDim, outputDim)
        activation_functions = [torch.nn.Identity, torch.nn.ReLU, torch.nn.Tanh, torch.nn.Sigmoid]
        activation_inputs = ["Identity", "ReLU", "Tanh", "Sigmoid"]
        self.activation = activation_functions[activation_inputs.index(activation)]()

        
    def forward(self, x):
        inp = self.linear(x)
        output = self.activation(inp)
        
        return output


In [27]:
class Neural_Network(torch.nn.Module):
    def __init__(self, layers):
        super(Neural_Network, self).__init__()
        self.layers = layers
        lst = []
        for layer in layers:
            lst += list(layer.parameters())
        self.params = torch.nn.ParameterList(lst)
        
    def forward(self, x):
        temp_input = x
        for layer in self.layers:
            temp_input = layer.forward(temp_input)
        return temp_input
    def predict(self, data):
        data_tensor = torch.FloatTensor(data)
        return [torch.argmax(self.forward(item)).item() for item in data_tensor]
    def score(self, x_test, y_test):
        x_test_tensor = torch.FloatTensor(pd.DataFrame(x_test).values)
        test_results = self.forward(x_test_tensor)
        results = [torch.argmax(res).item() for res in test_results]
        
        total = 0
        for i in range(len(results)):
            if results[i] == y_test[i]:
                total += 1

        return total/len(y_test)

    

In [28]:
def nn_construct(layer_dims, activation):
    
    return Neural_Network([Layer(layer_dims[i-1], layer_dims[i], activation) for i in range(1, len(layer_dims))])


In [29]:
import pandas as pd
import numpy as np
def nn_trainer(model, X_train, y_train):
    # Trains a model
    # Input: model: the neural network to be trained (Neural_Network)
    #        X_train: training for features
    #        y_train: training data for target
    criterion = torch.nn.CrossEntropyLoss()

    learning_rate = 1e-4
    optimizer = torch.optim.Adam(model.params, lr=learning_rate)

    for t in range(5000):
        # Forward pass: compute predicted y by passing x to the model.
        x_train_tensor = torch.FloatTensor(X_train)
        
        y_pred = model.forward(x_train_tensor)
        
        # Compute and print loss.
        loss = criterion(y_pred, torch.tensor(pd.DataFrame(y_train).values, dtype = torch.long).reshape(-1))

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
    
    return model.parameters(), model.forward(x_train_tensor)

In [38]:
# Constructs neural networks
relu_model = nn_construct([96, 48, 4], "ReLU")
sigmoid_model = nn_construct([96, 48, 4], "Sigmoid")

In [39]:
# Trains neural networks
relu_params, relu_activations = nn_trainer(relu_model, X_train.values, y_train)
sigmoid_params, sigmoid_activations = nn_trainer(sigmoid_model, X_train.values, y_train)

In [40]:
# Saves results
relu_test_score = relu_model.score(X_test,y_test)
sigmoid_test_score = sigmoid_model.score(X_test,y_test)


In [41]:
# Saves results
relu_train_score = relu_model.score(X_train, y_train)
sigmoid_train_score = sigmoid_model.score(X_train, y_train)

In [42]:
# Saves results
relu_val_score = relu_model.score(X_val, y_val)
sigmoid_val_score = sigmoid_model.score(X_val, y_val)

In [43]:
# Saves results
relu_scores = [relu_train_score, relu_val_score, relu_test_score]
sigmoid_scores = [sigmoid_train_score, sigmoid_val_score, sigmoid_test_score]

In [45]:
# Saves a table for classification results
summary = pd.DataFrame(data = {"RBF Kernel SVM": kernel_scores, "Linear SVM":linear_scores, "Descision Tree": dt_scores, "Neural Network (ReLU Activation)": relu_scores, "Neural Network (Sigmoid Activation)": sigmoid_scores})
summary.index = ["Accuracy on training", "Accuracy on validation", "Accuracy on testing"]
summary.T.to_html("tables/classification.html")